In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sympy import false
from sympy.abc import alpha
from tensorflow.python.ops.linalg.linear_operator_algebra import inverse

from datasetLoader import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from inverse import fit_linear
from tools import model_tester
from tools.reportParser import find_nearest
%matplotlib qt

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [35]:
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
with open("data/initial kernels/Kernel_Silica_Adsorption.npy", 'rb') as f:
    data_sorb = np.load(f)

x, y = load_dataset('data/datasets/silica_random.npz')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)

x_exp, y_exp = load_dataset('data/datasets/exp.npz')

x_train_exp, x_test_exp, y_train_exp, y_test_exp = train_test_split(x_exp, y_exp, test_size=0.15, random_state=1)

In [19]:
figure, axis = plt.subplots(3, 4)
for i in range(3):
    for j in range(4):
        k = np.random.randint(0, len(x_train_exp))
        axis[i, j].plot(pressures[:-10], x_train_exp[k], marker=".")
        axis[i, j].grid()
plt.show()

In [20]:
figure, axis = plt.subplots(3, 4)
for i in range(3):
    for j in range(4):
        k = np.random.randint(0, len(y_train_exp))
        axis[i, j].plot(pore_widths, y_train_exp[k], marker=".")
        axis[i, j].grid()
plt.show()

In [21]:
figure, axis = plt.subplots(3, 4)
for i in range(3):
    for j in range(4):
        k = np.random.randint(0, len(x_train))
        axis[i, j].plot(pore_widths, y_train[k], marker=".")
        axis[i, j].grid()
plt.show()

In [6]:
plt.plot(pore_widths, sum(y_train), marker=".")

In [7]:
i = np.random.randint(0, len(x_train))
plt.plot(pressures[:-10], x_train[i], marker=".")
plt.grid()
plt.show()

In [36]:
class IsothermDataset(Dataset):
    def __init__(self, isotherms, transform=None):
        self.data = torch.tensor(isotherms, dtype=torch.float32).to(device)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        if self.transform:
            x = self.transform(x)
        return x, x

x_mixed_train = np.concatenate((x_train_exp, x_train))
x_mixed_test = np.concatenate((x_test_exp, x_test))

# dataset = IsothermDataset(np.concatenate((x_train_exp, x_train_exp)))
# dataset_test = IsothermDataset(np.concatenate((x_test_exp, x_test_exp)))
dataset = IsothermDataset(np.concatenate((x_mixed_train, x_mixed_train)))
dataset_test = IsothermDataset(np.concatenate((x_mixed_test, x_mixed_test)))


batch_size = 512
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [37]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z

input_dim = 448
latent_dim = 16
epochs = 200
learning_rate = 1e-3

model = Autoencoder(input_dim=input_dim, latent_dim=latent_dim)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

def train_autoencoder(model, loader, loader_test):
    model.train()
    total_loss = 0
    total_vloss = 0
    for x, _ in loader:
        optimizer.zero_grad()
        x_recon, _ = model(x)
        loss = criterion(x_recon, x)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for x, _ in loader_test:
            x_recon, _  = model(x)
            vloss = criterion(x_recon, x)
            total_vloss += vloss.item()

    return total_loss / len(loader.dataset), total_vloss / len(loader_test.dataset)


# sample_z = model.encoder(torch.tensor(isotherms_np[0], dtype=torch.float32))

In [38]:
for epoch in range(1, epochs+1):
    loss, vloss = train_autoencoder(model, loader,loader_test)
    if epoch % 1 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss*100:.8f} Test loss: {vloss*100:.8f}")

Epoch 1/200, Loss: 0.00295697 Test loss: 0.00021172
Epoch 2/200, Loss: 0.00015777 Test loss: 0.00011480
Epoch 3/200, Loss: 0.00009958 Test loss: 0.00009277
Epoch 4/200, Loss: 0.00007486 Test loss: 0.00006260
Epoch 5/200, Loss: 0.00005777 Test loss: 0.00004983
Epoch 6/200, Loss: 0.00004702 Test loss: 0.00004064
Epoch 7/200, Loss: 0.00003855 Test loss: 0.00004000
Epoch 8/200, Loss: 0.00003495 Test loss: 0.00002958
Epoch 9/200, Loss: 0.00003093 Test loss: 0.00003215
Epoch 10/200, Loss: 0.00002823 Test loss: 0.00002556
Epoch 11/200, Loss: 0.00002570 Test loss: 0.00002458
Epoch 12/200, Loss: 0.00002357 Test loss: 0.00001912
Epoch 13/200, Loss: 0.00002128 Test loss: 0.00002003
Epoch 14/200, Loss: 0.00001957 Test loss: 0.00002058
Epoch 15/200, Loss: 0.00001914 Test loss: 0.00001791
Epoch 16/200, Loss: 0.00001730 Test loss: 0.00001649
Epoch 17/200, Loss: 0.00001764 Test loss: 0.00001296
Epoch 18/200, Loss: 0.00001698 Test loss: 0.00001385
Epoch 19/200, Loss: 0.00001484 Test loss: 0.00001414
Ep

KeyboardInterrupt: 

In [39]:
torch.save(model, "data/models/torch/autoencoder_exp.pt")

In [40]:
model = torch.load("data/models/torch/autoencoder_exp.pt", weights_only=False)
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=448, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=16, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=448, bias=True)
  )
)

In [44]:
model.eval()
latent_vectors_train = model.encoder(torch.tensor(x_train, dtype=torch.float32).to(device)).detach().cpu().numpy()
latent_vectors_test = model.encoder(torch.tensor(x_test, dtype=torch.float32).to(device)).detach().cpu().numpy()
latent_vectors_test_exp = model.encoder(torch.tensor(x_test_exp, dtype=torch.float32).to(device)).detach().cpu().numpy()

In [45]:
decoded = model.decoder(model.encoder(torch.tensor(x_test_exp, dtype=torch.float32).to(device))).detach().cpu().numpy()

In [29]:
figure, axis = plt.subplots(3, 3)
for i in range(3):
    for j in range(3):
        k=np.random.randint(0, len(decoded))
        axis[i, j].plot(pressures[:-10], x_test_exp[k], marker=".", label = "origin")
        axis[i, j].plot(pressures[:-10], decoded[k], marker=".", label = "decoded")
        axis[i, j].grid(True)
axis[i, j].legend()
plt.show()


# k=np.random.randint(0, len(decoded))
# plt.plot(pressures[:-10], x_test_exp[k], marker=".", label = "origin")
# plt.plot(pressures[:-10], decoded[k], marker=".", label = "decoded")
# plt.legend()
# plt.grid(True)
# plt.show()

In [14]:
np.random.seed(0)
labels = None 

pca = PCA(n_components=2)
latent_pca = pca.fit_transform(latent_vectors_train[:100])
latent_pca_exp = pca.fit_transform(latent_vectors_test[:100])

tsne = TSNE(n_components=2, init='pca', random_state=0)
latent_tsne = tsne.fit_transform(latent_vectors_train[:100])
latent_tsne_exp = tsne.fit_transform(latent_vectors_test[:100])

plt.figure()
plt.scatter(latent_pca[:, 0], latent_pca[:, 1], label="train")
plt.scatter(latent_pca_exp[:, 0], latent_pca_exp[:, 1], label="exp")
plt.title("PCA of Latent Space")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

plt.figure()
plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], label="train")
plt.scatter(latent_tsne_exp[:, 0], latent_tsne_exp[:, 1], label="exp")
for i in range(latent_tsne_exp.shape[0]):
        plt.text(latent_tsne_exp[i, 0], latent_tsne_exp[i, 1], str(i), fontsize=8, ha='center', va='center')
plt.title("t-SNE of Latent Space")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.legend()
plt.show()

C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(
C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
C:\Users\anton\PycharmProjects\isotherm\.venv\lib\site-packages\sklearn\manifold\_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [136]:
def find_exact_idx_last(array, value):
    flag = False
    for i in range(len(array)):
        if array[i] == value:
            flag = True
        if flag and array[i] != value:
            return i
def find_alpha(x_exp):
    def triangle_method(log_residuals, log_solution_norms):
        points = np.column_stack((log_residuals, log_solution_norms))
        A = points[0]
        B = points[-1]
        AB = B - A
        AB_norm = np.linalg.norm(AB)
        dist = []
        for i in range(0, len(points)):
            P = points[i]
            AP = P - A
            dist.append(np.linalg.norm(np.cross(AB, AP)) / AB_norm)
    
        return dist
    def calculate_roughness(psd, ord=2):
        return np.linalg.norm(psd, ord=ord)
    
    alpha_list = np.logspace(-5, 0, 40)
    error_lst = []
    roughness_lst = []
    restored_isotherms = []
    for alpha in alpha_list:
        start_idx = find_exact_idx_last(x_exp, 0)
        y = fit_linear(x_exp[start_idx:], kernel=data_sorb[:, start_idx:-10], alpha=alpha).x
        restored_isotherm = np.dot(y, data_sorb)
        restored_isotherms.append(restored_isotherm)
        error_lst.append(np.linalg.norm((x_exp[start_idx:] - restored_isotherm[start_idx:-10]), ord=2))
        roughness_lst.append(calculate_roughness(y))

    dist = triangle_method(np.log(error_lst), np.log(roughness_lst))
    alpha = alpha_list[np.argmax(dist)]
    return alpha



def plot_preds(x, y, preds): 
    NX, NY = 3, 4
    figure, axis = plt.subplots(NX, NY)
    for i in range(NX):
        for j in range(NY):
            
            low_p = False
            while low_p == False:
                k = np.random.randint(0, len(preds))
                for l in range(len(x[k])):
                    if x[k][l]!=0:
                        if pressures[l] < 1e-2:
                            low_p = True
                        break
            
            iso_axis = axis[i, j].twiny()
            iso_axis.set_xlabel("P/P$^0$",fontsize=8)
            iso_axis.plot(pressures[:-10], x[k], label="Isotherm", color = 'green')
            kernel = (data_sorb.T[:-10])
            iso_axis.plot(pressures[:-10], np.dot(kernel, preds[k][:128]), label="Isotherm by model", color="red")
            axis[i, j].set_title(f"№ {k}")
            axis[i, j].title.set_size(10)
            axis[i, j].grid()
            axis[i, j].set_xlabel("nm",fontsize=8)
            axis[i, j].plot(pore_widths, (preds[k]), marker=".", label=f"Model PSD")
            #axis[i, j].plot(pore_widths, y[k], marker=".", label="PSD")
            alpha = find_alpha(x[k])
            start_idx = find_exact_idx_last(x[k], 0)
            L_curve = fit_linear(x[k][start_idx:], kernel=data_sorb[:, start_idx:-10], alpha=alpha).x
            axis[i, j].plot(pore_widths, L_curve, marker=".", label="L_curve")
            iso_axis.plot(pressures[:-10], np.dot(kernel, L_curve), label="Isotherm by L_curve", color="yellow")
    plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.9)
    plt.legend()
    axis[0, 0].legend()
    plt.show()

In [56]:
from tools import model_tester
from inverse import fit_linear

error_lst, roughness_lst = model_tester.test_model_predictions(preds, x_test_exp, kernel=data_sorb[:, :-10])
kde_x, kde_error, kde_fun = model_tester.calculate_kde_data(error_lst, stop=150)
print("average error:", np.mean(error_lst))
plt.plot(kde_x, kde_error, label=model_name)
plt.grid(True)
plt.legend()
plt.plot()

average error: 19.576227837865584


[]

In [30]:
class DynamicWeightAveraging:
    def __init__(self, num_tasks, T=2.0):
        self.num_tasks = num_tasks
        self.T = T
        self.loss_history = []  # список списков: [ [L1_1, L1_2, ...], [L2_1, L2_2, ...], ... ]

    def update_weights(self):
        if len(self.loss_history[0]) < 2:
            return np.ones(self.num_tasks) / self.num_tasks

        r = []
        for i in range(self.num_tasks):
            li = self.loss_history[i]
            r_i = li[-1] / (li[-2] + 1e-8)
            r.append(r_i)

        r = np.array(r)
        weights = self.T * np.exp(r / self.T)
        weights /= weights.sum()
        return weights

    def append_losses(self, losses):  # losses — список текущих значений потерь [L1, L2, ...]
        if not self.loss_history:
            self.loss_history = [[] for _ in range(len(losses))]
        for i, l in enumerate(losses):
            self.loss_history[i].append(l)
dwa = DynamicWeightAveraging(num_tasks=2)

In [58]:
dwa.update_weights()

array([0.49557976, 0.50442024])

In [61]:
class PSD_model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PSD_model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, output_dim),
            nn.ReLU()
        )

    def forward(self, x):
        psd = self.model(x)
        return psd

class Isotherm_PSD_Dataset(Dataset):
    def __init__(self, x, y, original_x, transform=None):
        self.x = torch.tensor(x, dtype=torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.float32).to(device)
        self.original_x = torch.tensor(original_x, dtype=torch.float32).to(device)
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        original_x = self.original_x[idx]
        if self.transform:
            x = self.transform(x)
        return x, y, original_x

train_PSD = Isotherm_PSD_Dataset(latent_vectors_train, (y_train), x_train)
test_PSD = Isotherm_PSD_Dataset(latent_vectors_test, (y_test), x_test)

batch_size = 512
PSD_loader = DataLoader(train_PSD, batch_size=batch_size, shuffle=True)
PSD_loader_test = DataLoader(test_PSD, batch_size=batch_size, shuffle=False)


model_PSD = PSD_model(input_dim=latent_dim, output_dim=128)
model_PSD.to(device)

optimizer = optim.Adam(model_PSD.parameters(), lr=learning_rate)
criterion = nn.MSELoss()


with open("data/initial kernels/Kernel_Silica_Adsorption.npy", 'rb') as f:
    data_sorb_torch = torch.tensor(np.load(f)[:, :-10])
    data_sorb_torch = data_sorb_torch.to(torch.float32).to(device)

def isoterm_loss(predicted_y, x):
    restored_isotherm = torch.matmul(predicted_y, data_sorb_torch)
    loss = torch.mean((x - restored_isotherm) ** 2)
    return loss

def train_PSD_model(model, loader, loader_test):
    model.train()
    total_loss = 0
    total_vloss = 0
    for x, y, original_x in loader:
        optimizer.zero_grad()
        y_recon = model(x)
        loss = criterion(y_recon, y)
        iso_loss = isoterm_loss(y_recon, original_x)
        dwa.append_losses([loss.item(), iso_loss.item()])
        weights = dwa.update_weights()
        loss = weights[0] * loss + weights[1] * iso_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for x, y, original_x in loader_test:
            y_recon  = model(x)
            loss = criterion(y_recon, y)
            iso_loss = isoterm_loss(y_recon, original_x)
            vloss = weights[0] * loss + weights[1] * iso_loss
            total_vloss += vloss.item()

    return total_loss / len(loader.dataset), total_vloss / len(loader_test.dataset)


In [62]:
epochs = 100
loss_lst = []
vloss_lst = []
for epoch in range(1, epochs+1):
    loss, vloss = train_PSD_model(model_PSD, PSD_loader, PSD_loader_test)
    loss_lst.append(loss)
    vloss_lst.append(vloss)
    if epoch % 1 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss*100:.8f} Test loss: {vloss*100:.8f}")

Epoch 1/100, Loss: 0.00383810 Test loss: 0.00195509
Epoch 2/100, Loss: 0.00195923 Test loss: 0.00162222
Epoch 3/100, Loss: 0.00167126 Test loss: 0.00141452
Epoch 4/100, Loss: 0.00158459 Test loss: 0.00142282
Epoch 5/100, Loss: 0.00148877 Test loss: 0.00129145
Epoch 6/100, Loss: 0.00139659 Test loss: 0.00116572
Epoch 7/100, Loss: 0.00137226 Test loss: 0.00118691
Epoch 8/100, Loss: 0.00131694 Test loss: 0.00122773
Epoch 9/100, Loss: 0.00131264 Test loss: 0.00101837
Epoch 10/100, Loss: 0.00126100 Test loss: 0.00111655
Epoch 11/100, Loss: 0.00121896 Test loss: 0.00105156
Epoch 12/100, Loss: 0.00120544 Test loss: 0.00109989
Epoch 13/100, Loss: 0.00117798 Test loss: 0.00100141
Epoch 14/100, Loss: 0.00119106 Test loss: 0.00094200
Epoch 15/100, Loss: 0.00113441 Test loss: 0.00100388
Epoch 16/100, Loss: 0.00113051 Test loss: 0.00114247
Epoch 17/100, Loss: 0.00113529 Test loss: 0.00112608
Epoch 18/100, Loss: 0.00111494 Test loss: 0.00107649
Epoch 19/100, Loss: 0.00112374 Test loss: 0.00104921
Ep

In [52]:
plt.plot(loss_lst)
plt.plot(vloss_lst)
plt.show()

In [127]:
model_name = "autoencoder_regressor_combined"
torch.save(model_PSD, f"data/models/torch/{model_name}") 

In [13]:
model_name = "autoencoder_regressor"
model_PSD = torch.load(f"data/models/torch/{model_name}", weights_only=False)

In [63]:
model_PSD.eval()
y_train_PSD = model_PSD.model(torch.tensor(latent_vectors_train, dtype=torch.float32).to(device)).detach().cpu().numpy()
y_test_PSD = model_PSD.model(torch.tensor(latent_vectors_test, dtype=torch.float32).to(device)).detach().cpu().numpy()
y_test_exp_PSD = model_PSD.model(torch.tensor(latent_vectors_test_exp, dtype=torch.float32).to(device)).detach().cpu().numpy()

In [117]:
plot_preds(x_test_exp, y_test_exp, y_test_exp_PSD)

In [131]:
np.savez(f"data/models/metrics/{model_name}", x=x_test_exp, y=y_test_exp_PSD)
model_name


'autoencoder_regressor_combined'

In [72]:
model2 = "autoencoder_regressor_combined"
model2_data = np.load(f"data/models/metrics/{model2}.npz")
model2_x = model2_data["x"]
model2_y = model2_data["y"]

In [73]:

model2_y_math = [fit_linear(model2_x[i], data_sorb[:, :-10], 0).x for i in range(len(model2_x))]

In [137]:
plot_preds(model2_x, model2_y_math, model2_y)

In [83]:
import pandas as pd
data = pd.read_csv("data/initial kernels/excel/kernel_N2_77K_2koza.csv")

In [98]:
pressures = data["# P/P0\\H[nm]"]

0      1.268479e-09
1      1.596543e-09
2      2.009431e-09
3      2.529027e-09
4      3.183027e-09
           ...     
190    9.925630e-01
191    9.941513e-01
192    9.957396e-01
193    9.973292e-01
194    9.989173e-01
Name: # P/P0\H[nm], Length: 195, dtype: float64

In [126]:
a_array = []
for i in (data.columns[1:]).to_numpy():
    a_array.append(float(i))
a_array = np.array(a_array)

In [132]:
new_kernel = data.to_numpy()

In [133]:
np.save("data/initial kernels/new_kernel/pressure.npy", pressures)
np.save("data/initial kernels/new_kernel/pore_sizes.npy", a_array)
np.save("data/initial kernels/new_kernel/kernel.npy", new_kernel)

In [138]:
data 

,# P/P0\H[nm],0.7,0.7125,0.725,0.7375,0.75,0.7625,0.775,0.7875,0.8,...,48.0,50.0,55.0,60.0,65.0,70.0,75.0,80.0,90.0,100.0
0,1.268479e-09,3.952327,3.731933,3.321363,2.863562,2.381306,1.914137,1.479571,1.141257,0.876086,...,0.000332,0.000319,0.000290,0.000266,0.000245,0.000228,0.000212,0.000199,0.000177,0.000159
1,1.596543e-09,4.199571,3.943151,3.553391,3.067191,2.574083,2.027395,1.598272,1.233990,0.946521,...,0.000383,0.000367,0.000334,0.000306,0.000282,0.000262,0.000245,0.000229,0.000204,0.000183
2,2.009431e-09,4.423346,4.192714,3.780905,3.265074,2.716655,2.194424,1.744386,1.338909,1.023296,...,0.000482,0.000463,0.000420,0.000385,0.000356,0.000330,0.000308,0.000289,0.000257,0.000231
3,2.529027e-09,4.691274,4.406331,3.926953,3.426152,2.883507,2.367929,1.866774,1.438704,1.099539,...,0.000562,0.000539,0.000490,0.000449,0.000414,0.000385,0.000359,0.000336,0.000299,0.000269
4,3.183027e-09,4.942917,4.662738,4.235102,3.658560,3.120700,2.524314,2.017452,1.560537,1.210825,...,0.000677,0.000650,0.000591,0.000541,0.000500,0.000464,0.000433,0.000406,0.000360,0.000324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,9.925630e-01,15.390863,14.933137,14.587888,14.248162,13.888123,13.531749,13.305604,12.982052,12.763135,...,17.311887,17.311887,17.311887,17.311887,17.311887,17.311887,17.311887,17.311887,17.311887,17.311887
191,9.941513e-01,15.379747,14.929881,14.587508,14.257587,13.900280,13.541826,13.308271,13.003020,12.756473,...,17.295862,17.295862,17.295862,17.295862,17.295862,17.295862,17.295862,17.295862,17.295862,17.295862
192,9.957396e-01,15.391051,14.943118,14.597172,14.266703,13.898679,13.542538,13.311109,12.992426,12.759832,...,17.294825,17.294825,17.294825,17.294825,17.294825,17.294825,17.294825,17.294825,17.294825,17.294825
193,9.973292e-01,15.399291,14.961349,14.586886,14.267135,13.899183,13.522256,13.298693,13.010123,12.778168,...,17.309949,17.309949,17.309949,17.309949,17.309949,17.309949,17.309949,17.309949,17.309949,17.309949
